# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [2]:
df.info()
# Your code here

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We'll make a first selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [5]:
# Load necessary packages
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# remove "object"-type features and SalesPrice from `X`
features = [col for col in df.columns if df[col].dtype in [np.float64, np.int64] and col!='SalePrice']
X = df[features]


In [6]:
features[:4]  # List

['Id', 'MSSubClass', 'LotFrontage', 'LotArea']

In [7]:
# Impute null values by median value
for col in X:
    med = X[col].median()
    X[col].fillna(value = med, inplace = True)

# Create y
y = df.SalePrice

Look at the information of `X` again

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [9]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X,y)

# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('Training r^2:', linreg.score(X_train, y_train))
print('Testing r^2:', linreg.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, linreg.predict(X_test)))

Training r^2: 0.812518300705032
Testing r^2: 0.7934155783837109
Training MSE: 1119183658.9209363
Testing MSE: 1508962798.1064222


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [10]:
from sklearn import preprocessing

# scale the data and perform train test split
X_scaled = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled,y)

Perform the same linear regression on this data and print out R-squared and MSE.

In [13]:
linreg_norm = LinearRegression()
linreg_norm.fit(X_train, y_train)
print('Training r^2:', linreg_norm.score(X_train, y_train))
print('Testing r^2:', linreg_norm.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg_norm.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, linreg_norm.predict(X_test)))

Training r^2: 0.8708447299221075
Testing r^2: 0.5801248275580315
Training MSE: 777200720.2869304
Testing MSE: 3011822573.284284


## Include dummy variables

We haven't included dummy variables so far: let's use our "object" variables again and create dummies

In [14]:
# Create X_cat which contains only the categorical variables
features_cat = [col for col in df.columns if df[col].dtype in [np.object]]
X_cat = df[features_cat]

np.shape(X_cat)

(1460, 43)

In [16]:
features_cat[:5]

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour']

In [17]:
# Make dummies
X_cat = pd.get_dummies(X_cat)
np.shape(X_cat)

(1460, 252)

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [20]:
# Your code here
X_all = pd.concat([pd.DataFrame(X_scaled), X_cat], axis = 1)

Perform the same linear regression on this data and print out R-squared and MSE.

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y)
linreg_all = LinearRegression()
linreg_all.fit(X_train, y_train)
print('Training r^2:', linreg_all.score(X_train, y_train))
print('Testing r^2:', linreg_all.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg_all.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, linreg_all.predict(X_test)))

Training r^2: 0.9195194402227551
Testing r^2: -1.186260443467979e+21
Training MSE: 488581243.5972603
Testing MSE: 8.320118945451793e+30


Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far far off. Similarly, the scale of the Testing MSE is orders of magnitude higher then that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [26]:
from sklearn.linear_model import Lasso, Ridge

lasso = Lasso() 
lasso.fit(X_train, y_train)
print('Training r^2:', lasso.score(X_train, y_train))
print('Testing r^2:', lasso.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, lasso.predict(X_test)))

Training r^2: 0.9380001585347971
Testing r^2: 0.6971585601708001
Training MSE: 376388530.7177819
Testing MSE: 2124050257.9894373


With a higher regularization parameter (alpha = 10)

In [27]:
from sklearn.linear_model import Lasso, Ridge

lasso = Lasso(alpha=10) #Lasso is also known as the L1 norm.
lasso.fit(X_train, y_train)
print('Training r^2:', lasso.score(X_train, y_train))
print('Testing r^2:', lasso.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, lasso.predict(X_test)))

Training r^2: 0.9365722005438225
Testing r^2: 0.7149622321160438
Training MSE: 385057375.62847954
Testing MSE: 1999179982.607766


## Ridge

With default parameter (alpha = 1)

In [29]:
from sklearn.linear_model import Lasso, Ridge

ridge = Ridge() #Ridge is also known as the L2 norm.
ridge.fit(X_train, y_train)
print('Training r^2:', ridge.score(X_train, y_train))
print('Testing r^2:', ridge.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, ridge.predict(X_test)))

Training r^2: 0.9350660355660524
Testing r^2: 0.7181170174216105
Training MSE: 394200999.3798337
Testing MSE: 1977053147.7004638


With default parameter (alpha = 10)

In [30]:
from sklearn.linear_model import Lasso, Ridge

ridge = Ridge(alpha = 10) #Lasso is also known as the L1 norm.
ridge.fit(X_train, y_train)
print('Training r^2:', ridge.score(X_train, y_train))
print('Testing r^2:', ridge.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, ridge.predict(X_test)))

Training r^2: 0.9230809045194533
Testing r^2: 0.7422803098962325
Training MSE: 466960312.28261405
Testing MSE: 1807578165.533091


## Look at the metrics, what are your main conclusions?

Conclusions here

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

In [31]:
# number of Ridge params almost zero
print(sum(abs(ridge.coef_) < 10**(-10)))

6


In [32]:
# number of Lasso params almost zero
print(sum(abs(lasso.coef_) < 10**(-10)))

64


In [33]:
lasso.coef_

array([-4.71412854e+02, -1.93761406e+03,  1.31204878e+03,  4.71691529e+03,
        9.27877317e+03,  6.43123698e+03,  9.90572581e+03,  2.37874642e+03,
        2.95557656e+03,  7.82017436e+03,  1.98573064e+03,  2.87427932e+02,
        5.16296372e+03,  1.62331774e+04,  1.98952564e+04,  2.70148828e+02,
        5.55383727e+03,  1.78591414e+03,  7.52214732e+01,  1.26272531e+03,
       -2.88003408e+02, -5.74225820e+02, -1.98195453e+03, -5.79616549e+00,
        2.40871744e+03, -1.68717408e+03,  3.60656356e+03,  5.20029952e+03,
        2.75910986e+03,  8.29798969e+02,  1.50718291e+03,  1.02281104e+03,
        2.54859973e+03,  4.09039372e+03,  3.93679917e+01, -1.74823677e+03,
        1.82753144e+02, -2.16209416e+04,  1.23101221e+03, -0.00000000e+00,
        5.74390499e+02, -3.48380878e+03, -3.01309985e+04,  0.00000000e+00,
       -3.56132720e+02,  1.04842427e+03, -2.19133938e+03,  4.76060082e+03,
        7.77915321e+03, -2.00888232e+02, -7.34175285e+03,  1.52590784e+03,
       -1.07908677e+04,  

Compare with the total length of the parameter space and draw conclusions!

In [ ]:
# your code here

## Summary

Great! You now know how to perform Lasso and Ridge regression.